In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv('kakaotalk_export.csv')
df.describe()

,Date,User,Message
count,42825,42825,42825
unique,40914,6,31376
top,2017-01-16 20:15:39,박창수,ㄷㄷ
freq,5,13895,664


In [2]:
df['Message'] = df['Message'].map(lambda m: m.replace('\n', ''))

In [3]:
user = ''

cs_msg = list()
msg_bf = ''
for i, row in df.iterrows():
    if row['User'] == user:
        msg_bf = msg_bf + ' ' + row['Message']
    else:
        if not msg_bf == '':
            cs_msg.append(msg_bf.strip())
            msg_bf = ''

In [4]:
letter_dict = dict()
for msg in cs_msg:
    for l in msg:
        letter_dict[l] = letter_dict[l] + 1 if l in letter_dict else 1
        
letter_list = ['<<blank>>'] + list(letter_dict.keys())

In [5]:
numbered_message = list(map(lambda m: [letter_list.index(l) for l in m], cs_msg))
numbered_message = [[0] + msg + [0] for msg in numbered_message]

In [6]:
scan_size = 7
def batch(msgs, l_list):
    x = list()
    y = list()
    for msg in msgs:
        if len(msg) < scan_size + 1:
            x.append(msg + [0] * (scan_size - len(msg)))
            y.append(msg[1:] + [0] * (scan_size + 1 - len(msg)))
        else:
            for i in range(0, len(msg) - (scan_size + 1)):
                x.append(msg[i:i+scan_size])
                y.append(msg[i+1:i+scan_size+1])
    return x, y

x = np.matrix(batch(numbered_message, letter_list)[0])
y = np.matrix(batch(numbered_message, letter_list)[1])

In [7]:
def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

tf.reset_default_graph()

X = tf.placeholder(shape=[None, scan_size], dtype=tf.int32)
Y = tf.placeholder(shape=[None, scan_size], dtype=tf.int32)

batch_size = tf.placeholder(tf.int32)

train_oh = tf.one_hot(X, len(letter_list))
targets = Y

cell = tf.contrib.rnn.BasicLSTMCell(len(letter_list), forget_bias=0.0)
cells = tf.contrib.rnn.MultiRNNCell([cell for i in range(5)])
state = cell.zero_state(batch_size, dtype=tf.float32)

outputs, f_state = tf.nn.dynamic_rnn(cell, train_oh, initial_state=state, sequence_length=length(train_oh))

In [8]:
outputs = tf.reshape(outputs, [-1, len(letter_list)])

w1 = tf.get_variable(shape=[len(letter_list), len(letter_list)], name='weight1')
b1 = tf.get_variable(shape=[len(letter_list)], name='bias1')
y1 = tf.matmul(outputs, w1) + b1

w2 = tf.get_variable(shape=[len(letter_list), len(letter_list)], name='weight2')
b2 = tf.get_variable(shape=[len(letter_list)], name='bias2')
y2 = tf.matmul(y1, w2) + b2

y2 = tf.reshape(y2, [-1, scan_size, len(letter_list)])

weights = tf.ones([batch_size, scan_size])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=y2, targets=targets, weights=weights)
loss = tf.reduce_mean(sequence_loss)

optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [11]:
def list_to_seq(l, l_list):
    result = list()
    for i in l:
        result.append(l_list[i])
    
    return "'" + ''.join(result) + "'"

from tqdm import tqdm

for p in tqdm(range(100)):
    for i in range(int(len(x) / 100)):
        nowX = x[i * 100: (i+1) * 100]
        nowY = y[i * 100: (i+1) * 100]
        sess.run(optimize, feed_dict={X: nowX, Y: nowY, batch_size: len(nowX)})
    
    if p % 10 == 0:
        c_loss = sess.run(loss, feed_dict={X: nowX, Y: nowY, batch_size: len(nowX)})
        sample = sess.run(y2, feed_dict={X: x[998], batch_size: 1})
        sample_list = np.argmax(sample, axis=2).reshape(-1)
        print(c_loss, list_to_seq(sample_list, letter_list))


  1%|          | 1/100 [00:26<43:44, 26.51s/it]

1.32299 '잇   고 라'


 11%|█         | 11/100 [04:51<39:23, 26.56s/it]

0.49126 '있긴 먹고올까'


 21%|██        | 21/100 [09:15<34:57, 26.56s/it]

...

 81%|████████  | 81/100 [35:53<08:22, 26.47s/it]

0.403963 '있? 먹고올까'


 91%|█████████ | 91/100 [40:24<04:09, 27.72s/it]

0.404087 '있? 먹고올까'


100%|██████████| 100/100 [44:29<00:00, 26.96s/it]


In [12]:
def predict_next(seq):
    if len(seq) >= scan_size:
        seq = seq[-scan_size:]
    else:
        seq = seq + [0] * (scan_size-len(seq))
        
    predict = sess.run(y2, feed_dict={X: [seq], batch_size: 1})
    predict_list = np.argmax(predict, axis=2).reshape(-1)
    last = predict_list[-1 if len(seq) >= scan_size else len(seq) - 1]
    
    return last

In [24]:
start = '오늘 점심은 '
start = [letter_list.index(l) for l in start]

for i in range(20):
    start.append(predict_next(start))
    
list_to_seq(start, letter_list)

"'오늘 점심은 만들으면 되는거지?킼 아재 안먹지? '"